# DATA PREPARATION 

## Import Libraries

In [1]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


## Functions

In [2]:
def load_input_df():
    df00 = pd.read_csv('../input/df.csv')
    return df00

def load_metadata():
    metadata = pd.read_csv('../artifacts/metadata.csv')
    return metadata

def save_mean_vars_num(mean_vars_num):
    joblib.dump(mean_vars_num, "../artifacts/mean_vars_num.joblib")

def load_mean_vars_num():
    return joblib.load("../artifacts/mean_vars_num.joblib")

def save_features_used_to_train(features_used_to_train):
    joblib.dump(features_used_to_train, "../artifacts/features_used_to_train.joblib")

def load_features_used_to_train():
    return joblib.load("../artifacts/features_used_to_train.joblib")

def save_scaler(scaler):
    joblib.dump(scaler,"../artifacts/scaler.joblib")

def load_scaler():
    return joblib.load("../artifacts/scaler.joblib")

def split_train_test_val(df00):
    df00.set_index('id',inplace=True)
    data = df00.drop(columns=['target'])

    # Get the target variable
    answer = df00['target']

    # Split data into training, validation and test sets
    X_trainval, X_test, y_trainval, y_test = train_test_split(data, answer
                                                            , test_size=0.2
                                                            , stratify=df00['target']
                                                            , random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval
                                                    , test_size=0.25
                                                    , stratify=y_trainval
                                                    , random_state=42)
    
    return X_train, y_train, X_val, y_val, X_test, y_test


def data_prep(df00,metadata, istrain=False):
    # Filtering Data Frame 
    vars_numericas_df = metadata[((metadata.Level  == 'ordinal')|(metadata.Level == 'interval')) & (metadata.Role == 'input')]
    # list of selecting features
    lista_vars_numericas = list(vars_numericas_df['Features'])
    df01 = df00[lista_vars_numericas]
    ### 1)  Treating Missing Values
    # Verifying amount missing values
    df01_missing = df01.isnull().sum()
    # count rows of dataframe
    qt_rows = df00.shape[0]
    # dataframe with quantity of missing values by feature
    df_pct_missing = pd.DataFrame(df01_missing,columns=['qt_missing'])
    df_pct_missing = pd.DataFrame(df01_missing,columns=['qt_missing'])
    df_pct_missing['Features'] = df_pct_missing.index
    df_pct_missing['pc_miss'] = (100*df_pct_missing['qt_missing'].divide(qt_rows)).astype(int)
    df_pct_missing['qt_rows'] = qt_rows
    df_pct_missing.reset_index(drop = True, inplace = True)

    if istrain==True:
        ### Save mean to fill NaN values during deployment.
        mean_vars_num = df01[lista_vars_numericas].mean()
        save_mean_vars_num(mean_vars_num)
    if istrain==False:
        mean_vars_num = load_mean_vars_num()

    # Replace missing values by mean 
    df02 = (df01.fillna(mean_vars_num)).astype(float)

    # Categorical Features (String/Char) 
    ### 1) Selecting nominal features
    # Filter Data Frame 
    vars_char_df = metadata[(metadata.Level  == 'nominal') & (metadata.Role == 'input')]
    # list of cat features
    lista_vars_char = list(vars_char_df['Features'])
    ### Create a dataframe with only categorical features
    df06 = df00[lista_vars_char]
    ### 2) Verify missing values
    df06_missing = df06.isnull().sum()
    ### 3) Treat missing values (replace by MISS category)
    df07 = df06.fillna('MISS')
    
    df07.home_ownership.replace({"ANY":"OTHERS","OTHER":"OTHERS","NONE":"OTHERS"},inplace=True)
    
    ### 3) Create Dummies features
    # Filter Data Frame 
    vars_to_dummif_df = metadata[(metadata.Level  == 'nominal') & (metadata.Role == 'input')& (metadata.Cardinality <= 20)]
    # list of cat features
    lista_vars_dummif = list(vars_to_dummif_df['Features'])
    df08 = df07[lista_vars_dummif]
    # Generate dummies
    if istrain==True:
        df09 = pd.get_dummies(df08, 
                            columns=lista_vars_dummif,
                            drop_first=False, 
                            prefix = lista_vars_dummif,
                            prefix_sep='_',
                            dtype=int)
        features_used_to_train = df09.columns.tolist()
        
        save_features_used_to_train(features_used_to_train)

    if istrain==False:
        df09 = pd.get_dummies(df08, 
                            columns=lista_vars_dummif,
                            drop_first=False, 
                            prefix = lista_vars_dummif,
                            prefix_sep='_',
                            dtype=int)
        l1 = df09.columns.to_list()
        l2 = load_features_used_to_train()

        columns_to_add = list(set(l2)-set(l1))
        # Adicionando as novas colunas com valores padrão NaN
        for col in columns_to_add:
            df09[col] = 0

        df09 = df09[l2]

    
    ### 4) Join all treat dataframe 
    # * df02 --> non standalization
    # * df09 --> dummies

    df_expl_not_norm = pd.merge(df02,df09, left_index=True, right_index=True)
    return df_expl_not_norm


def normalization(df00,metadata,istrain=False):
    vars_numericas_df = metadata[((metadata.Level  == 'ordinal')|(metadata.Level == 'interval')) & (metadata.Role == 'input')]
    # list of selecting features
    lista_vars_numericas = list(vars_numericas_df['Features'])

    if istrain==True:
        scaler = StandardScaler()
        scaler.fit(df00[lista_vars_numericas])
        # After fitting the scaler to your data as you did:
        save_scaler(scaler)
    if istrain==False:
        scaler=load_scaler()

    df00_std = df00.copy()
    
    # Scale the training, validation, and test sets using the scaler
    df00_std[lista_vars_numericas] = scaler.transform(df00[lista_vars_numericas])
    return df00_std




## Load data

In [3]:

df00 = load_input_df()

df00.head()

,id,loan_amnt,term,emp_length,home_ownership,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl,target
0,68407277,3600.0,1.0,10.0,MORTGAGE,55000.0,45,5.91,4.0,1506.0,4.0,0
1,68355089,24700.0,1.0,10.0,MORTGAGE,65000.0,6,16.06,0.0,57830.0,20.0,0
2,68341763,20000.0,2.0,10.0,MORTGAGE,63000.0,47,10.78,10.0,2737.0,4.0,0
3,68476807,10400.0,2.0,3.0,MORTGAGE,104433.0,45,25.37,1.0,4567.0,7.0,0
4,68426831,11950.0,1.0,4.0,RENT,34000.0,43,10.20,NaN,844.0,4.0,0


## Load metadata

In [4]:
metadata = load_metadata()
metadata

,Features,Role,Level,Keep,Tipo,Cardinality
0,id,id,nominal,False,object,1345310
1,loan_amnt,input,interval,True,float64,1556
2,term,input,interval,True,float64,2
3,emp_length,input,interval,True,float64,11
4,home_ownership,input,nominal,True,object,6
5,annual_inc,input,interval,True,float64,64362
6,addr_state,input,ordinal,True,int64,51
7,dti,input,interval,True,float64,7067
8,mths_since_recent_inq,input,interval,True,float64,26
9,bc_open_to_buy,input,interval,True,float64,74924


## Split data into training, validation and test sets, ensuring the class distribution is maintained


In [5]:

X_train, y_train, X_val, y_val, X_test, y_test = split_train_test_val(df00)

In [6]:
X_train.head()

,loan_amnt,term,emp_length,home_ownership,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl
id,,,,,,,,,,
105253408,16000.0,1.0,7.0,MORTGAGE,84996.0,43,22.91,21.0,996.0,3.0
16592425,19200.0,2.0,NaN,OWN,62960.0,30,19.06,1.0,19930.0,17.0
58470345,6000.0,1.0,7.0,RENT,30000.0,51,6.48,3.0,0.0,3.0
10092824,10000.0,1.0,7.0,MORTGAGE,150000.0,34,10.81,11.0,122590.0,34.0
65965690,21000.0,1.0,10.0,MORTGAGE,144000.0,18,20.84,6.0,18590.0,7.0


## Data Preparation

In [7]:
X_train_not_norm = data_prep(X_train,metadata, istrain=True)
X_val_not_norm = data_prep(X_val,metadata, istrain=False)
X_test_not_norm = data_prep(X_test,metadata, istrain=False)

In [8]:

df_not_norm = data_prep(df00,metadata, istrain=False)

In [9]:
df_not_norm.head()

,loan_amnt,term,emp_length,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl,home_ownership_MORTGAGE,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT
id,,,,,,,,,,,,,
68407277,3600.0,1.0,10.0,55000.0,45.0,5.91,4.000000,1506.0,4.0,1,0,0,0
68355089,24700.0,1.0,10.0,65000.0,6.0,16.06,0.000000,57830.0,20.0,1,0,0,0
68341763,20000.0,2.0,10.0,63000.0,47.0,10.78,10.000000,2737.0,4.0,1,0,0,0
68476807,10400.0,2.0,3.0,104433.0,45.0,25.37,1.000000,4567.0,7.0,1,0,0,0
68426831,11950.0,1.0,4.0,34000.0,43.0,10.20,6.711641,844.0,4.0,0,0,0,1


In [10]:
X_train_not_norm.head()

,loan_amnt,term,emp_length,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl,home_ownership_MORTGAGE,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT
id,,,,,,,,,,,,,
105253408,16000.0,1.0,7.000000,84996.0,43.0,22.91,21.0,996.0,3.0,1,0,0,0
16592425,19200.0,2.0,5.965058,62960.0,30.0,19.06,1.0,19930.0,17.0,0,0,1,0
58470345,6000.0,1.0,7.000000,30000.0,51.0,6.48,3.0,0.0,3.0,0,0,0,1
10092824,10000.0,1.0,7.000000,150000.0,34.0,10.81,11.0,122590.0,34.0,1,0,0,0
65965690,21000.0,1.0,10.000000,144000.0,18.0,20.84,6.0,18590.0,7.0,1,0,0,0


In [11]:
X_train_not_norm.shape, X_val_not_norm.shape, X_test_not_norm.shape

((807186, 13), (269062, 13), (269062, 13))

## Data Normalization

In [12]:

X_train_std = normalization(X_train_not_norm,metadata,istrain=True)
X_val_std = normalization(X_val_not_norm,metadata,istrain=False)
X_test_std = normalization(X_test_not_norm,metadata,istrain=False)

In [13]:
X_train_std.head()

,loan_amnt,term,emp_length,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl,home_ownership_MORTGAGE,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT
id,,,,,,,,,,,,,
105253408,0.179897,-0.564595,2.890743e-01,0.119112,0.231563,0.410550,2.617179,-0.612346,-1.188047,1,0,0,0
16592425,0.546808,1.771181,-4.961623e-16,-0.183949,-0.964507,0.069240,-1.046193,0.647717,1.963179,0,0,1,0
58470345,-0.966701,-0.564595,2.890743e-01,-0.637249,0.967607,-1.046003,-0.679856,-0.678630,-1.188047,0,0,0,1
10092824,-0.508062,-0.564595,2.890743e-01,1.013113,-0.596485,-0.662140,0.785493,7.479771,5.789667,1,0,0,0
65965690,0.753196,-0.564595,1.127018e+00,0.930595,-2.068572,0.227040,-0.130350,0.558540,-0.287696,1,0,0,0


## Bring the target to create Train, Val e Test dataset 

In [14]:
Train = X_train_std.copy()
Train['target'] = y_train

Val = X_val_std.copy()
Val['target'] = y_val

Test = X_test_std.copy()
Test['target'] = y_test

Train.shape, Val.shape, Test.shape


((807186, 14), (269062, 14), (269062, 14))

In [15]:
df_not_norm = pd.merge(df_not_norm,df00['target'], how='left',right_index=True,left_index=True)

In [16]:
Train.head()

,loan_amnt,term,emp_length,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl,home_ownership_MORTGAGE,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,target
id,,,,,,,,,,,,,,
105253408,0.179897,-0.564595,2.890743e-01,0.119112,0.231563,0.410550,2.617179,-0.612346,-1.188047,1,0,0,0,1
16592425,0.546808,1.771181,-4.961623e-16,-0.183949,-0.964507,0.069240,-1.046193,0.647717,1.963179,0,0,1,0,0
58470345,-0.966701,-0.564595,2.890743e-01,-0.637249,0.967607,-1.046003,-0.679856,-0.678630,-1.188047,0,0,0,1,0
10092824,-0.508062,-0.564595,2.890743e-01,1.013113,-0.596485,-0.662140,0.785493,7.479771,5.789667,1,0,0,0,0
65965690,0.753196,-0.564595,1.127018e+00,0.930595,-2.068572,0.227040,-0.130350,0.558540,-0.287696,1,0,0,0,1


## Save final data


In [17]:
# Vamos utilizar o formato csv porque esta tabela é pequena em linhas e colunas 
Train.to_csv('../input/Train.csv', sep=',', encoding='utf-8',index=True)
Val.to_csv('../input/Val.csv', sep=',', encoding='utf-8',index=True)
Test.to_csv('../input/Test.csv', sep=',', encoding='utf-8',index=True)

In [18]:
# df_not_norm.to_csv('../input/df_not_norm.csv', sep=',', encoding='utf-8',index=True)

In [19]:
Train.to_csv('../input/Train_reduzido.csv.zip', sep=',', encoding='utf-8', index=True, compression='zip')


In [20]:
# # # Comprimir o arquivo CSV (opcional)
# import zipfile
# with zipfile.ZipFile('../input/Train_reduzido.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
#     zipf.write('../input/Train_reduzido.csv', arcname='Train_reduzido.csv')